<a href="https://colab.research.google.com/github/Cinichi/Ani-Downloader/blob/main/anime_downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# 🎬 AnimeKai Episode Downloader & Merger
# Enhanced with chunk downloads, yt-dlp support, and better error handling

# @title 🔧 **Install Dependencies** { display-mode: "form" }
print("📦 Installing required packages...")
!pip install -q requests beautifulsoup4 cloudscraper m3u8 pycryptodome tqdm yt-dlp
!apt-get -qq install -y ffmpeg aria2 > /dev/null 2>&1
print("✅ All dependencies installed!\n")

# @title ⚙️ **Configuration** { display-mode: "form" }

#@markdown ### 🔗 Anime URL
#@markdown Enter the AnimeKai watch URL:
anime_url = "https://animekai.to/watch/jujutsu-kaisen-4gm6" #@param {type:"string"}

#@markdown ---
#@markdown ### 📺 Episode Selection
download_mode = "Episode Range" #@param ["All Episodes", "Episode Range", "Single Episode"]

#@markdown Single episode number:
single_episode = 1 #@param {type:"integer"}

#@markdown Episode range (Start and End):
start_episode = 1 #@param {type:"integer"}
end_episode = 3 #@param {type:"integer"}

#@markdown ---
#@markdown ### 🎥 Quality & Audio Settings
video_quality = "1080p" #@param ["1080p", "720p", "480p", "360p"]
prefer_type = "Soft Sub" #@param ["Hard Sub", "Soft Sub", "Dub & S-Sub"]
prefer_server = "Server 1" #@param ["Server 1", "Server 2"]

#@markdown ---
#@markdown ### 📥 Download Settings
download_method = "yt-dlp" #@param ["yt-dlp", "aria2", "chunks", "ffmpeg"]

#@markdown Chunk size in MB (for chunked downloads):
chunk_size_mb = 5 #@param {type:"slider", min:1, max:50, step:1}

#@markdown Max parallel workers/connections:
max_workers = 8 #@param {type:"slider", min:1, max:32, step:1}

#@markdown Max retry attempts:
max_retries = 5 #@param {type:"slider", min:1, max:10, step:1}

#@markdown Timeout in seconds:
timeout = 300 #@param {type:"slider", min:60, max:600, step:30}

#@markdown ---
#@markdown ### 🔗 Merge Settings
merge_episodes = False #@param {type:"boolean"}

#@markdown ---
#@markdown ### 📤 Upload Settings
upload_to = "None (Keep Local)" #@param ["Google Drive Only", "GoFile.io Only", "Both", "None (Keep Local)"]

print("✅ Configuration loaded!")

# @title 🌐 **Core Functions** { display-mode: "form" }

import requests
import re
import json
import os
import time
import subprocess
import threading
from bs4 import BeautifulSoup
import cloudscraper
from urllib.parse import urljoin, urlparse, quote, unquote
import shutil
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# Create cloudscraper session
scraper = cloudscraper.create_scraper(
    browser={'browser': 'chrome', 'platform': 'windows', 'desktop': True}
)

BASE_URL = "https://animekai.to"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
    'Referer': BASE_URL,
    'Accept': '*/*',
    'Accept-Language': 'en-US,en;q=0.9',
    'Connection': 'keep-alive'
}

def enc_dec_request(endpoint, text):
    """Make request to enc-dec API"""
    try:
        url = f"https://enc-dec.app/api/{endpoint}?text={text}"
        response = scraper.get(url, headers=headers, timeout=30)
        data = response.json()
        return data.get('result', '')
    except Exception as e:
        print(f"⚠️ Enc-dec error: {e}")
        return None

def get_anime_details(url):
    """Get anime ID and title"""
    try:
        response = scraper.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        anime_div = soup.select_one('div[data-id]')
        anime_id = anime_div.get('data-id') if anime_div else None

        title_elem = soup.select_one('div.title-wrapper h1.title span')
        title = title_elem.get('title', '') if title_elem else "Unknown"
        title = re.sub(r'[<>:"/\\|?*]', '', title)

        return anime_id, title
    except Exception as e:
        print(f"❌ Error getting anime details: {e}")
        return None, None

def get_episode_list(anime_id):
    """Get list of all episodes"""
    try:
        enc = enc_dec_request('enc-kai', anime_id)
        if not enc:
            return []

        ep_url = f"{BASE_URL}/ajax/episodes/list?ani_id={anime_id}&_={enc}"
        response = scraper.get(ep_url, headers=headers)
        data = response.json()

        if 'result' not in data:
            return []

        html = data['result']
        soup = BeautifulSoup(html, 'html.parser')

        episodes = []
        for ep in soup.select('div.eplist a'):
            token = ep.get('token', '')
            ep_num = ep.get('num', '0')
            langs = ep.get('langs', '0')

            langs_int = int(langs) if langs.isdigit() else 0
            if langs_int == 1:
                subdub = "Sub"
            elif langs_int == 3:
                subdub = "Dub & Sub"
            else:
                subdub = ""

            episodes.append({
                'number': float(ep_num),
                'token': token,
                'subdub': subdub,
                'title': f"Episode {ep_num}"
            })

        return sorted(episodes, key=lambda x: x['number'])
    except Exception as e:
        print(f"❌ Error getting episodes: {e}")
        return []

def get_video_servers(token):
    """Get available video servers"""
    try:
        enc = enc_dec_request('enc-kai', token)
        if not enc:
            return []

        url = f"{BASE_URL}/ajax/links/list?token={token}&_={enc}"
        response = scraper.get(url, headers=headers)
        data = response.json()

        if 'result' not in data:
            return []

        html = data['result']
        soup = BeautifulSoup(html, 'html.parser')

        servers = []
        for type_div in soup.select('div.server-items[data-id]'):
            type_id = type_div.get('data-id', '')

            for server in type_div.select('span.server[data-lid]'):
                server_id = server.get('data-lid', '')
                server_name = server.text.strip()

                servers.append({
                    'type': type_id,
                    'server_id': server_id,
                    'server_name': server_name
                })

        return servers
    except Exception as e:
        print(f"❌ Error getting servers: {e}")
        return []

def get_video_url(server_id, server_name):
    """Get direct video URL"""
    try:
        enc = enc_dec_request('enc-kai', server_id)
        if not enc:
            return None

        url = f"{BASE_URL}/ajax/links/view?id={server_id}&_={enc}"
        response = scraper.get(url, headers=headers)
        data = response.json()

        encoded_link = data.get('result', '')
        if not encoded_link:
            return None

        dec_body = json.dumps({"text": encoded_link})
        dec_response = scraper.post(
            "https://enc-dec.app/api/dec-kai",
            data=dec_body,
            headers={'Content-Type': 'application/json'}
        )
        dec_data = dec_response.json()
        iframe_url = dec_data.get('result', {}).get('url', '')

        if not iframe_url:
            return None

        return extract_megaup_url(iframe_url)
    except Exception as e:
        print(f"⚠️ Error getting video URL: {e}")
        return None

def extract_megaup_url(iframe_url):
    """Extract video URL from MegaUp"""
    try:
        parsed = urlparse(iframe_url)
        token = parsed.path.split('/')[-1]

        media_url = f"{parsed.scheme}://{parsed.netloc}/media/{token}"
        response = scraper.get(media_url, headers=headers)
        data = response.json()
        mega_token = data.get('result', '')

        if not mega_token:
            return None

        dec_body = json.dumps({"text": mega_token, "agent": headers['User-Agent']})
        dec_response = scraper.post(
            "https://enc-dec.app/api/dec-mega",
            data=dec_body,
            headers={'Content-Type': 'application/json'}
        )

        mega_data = dec_response.json()
        sources = mega_data.get('result', {}).get('sources', [])

        if not sources:
            return None

        return sources[0].get('file', '')
    except Exception as e:
        print(f"⚠️ MegaUp extraction error: {e}")
        return None

print("✅ Core functions loaded")

# @title 📥 **Download Methods** { display-mode: "form" }

def download_with_ytdlp(url, output_file, episode_num):
    """Download using yt-dlp (BEST for m3u8)"""
    try:
        print(f"\n📥 Downloading Episode {episode_num} with yt-dlp...")
        print(f"   File: {os.path.basename(output_file)}")

        cmd = [
            'yt-dlp', url, '-o', output_file,
            '--no-warnings', '--no-check-certificate',
            '--concurrent-fragments', str(max_workers),
            '--retries', str(max_retries),
            '--fragment-retries', str(max_retries),
            '--socket-timeout', str(timeout),
            '--progress', '--newline',
            '--user-agent', headers['User-Agent'],
            '--referer', BASE_URL
        ]

        process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)

        for line in process.stdout:
            if '[download]' in line and '%' in line:
                match = re.search(r'(\d+\.\d+)%', line)
                if match:
                    percent = float(match.group(1))
                    print(f"\r   ⏳ Progress: {percent:.1f}%", end='', flush=True)

        process.wait()

        if process.returncode == 0 and os.path.exists(output_file):
            file_size = os.path.getsize(output_file) / (1024*1024)
            print(f"\n   ✅ Complete! ({file_size:.2f} MB)")
            return True
        else:
            print(f"\n   ❌ Download failed")
            return False
    except Exception as e:
        print(f"\n   ❌ Error: {e}")
        return False

def download_with_aria2(url, output_file, episode_num):
    """Download using aria2c (FAST multi-connection)"""
    try:
        print(f"\n📥 Downloading Episode {episode_num} with aria2...")
        print(f"   File: {os.path.basename(output_file)}")

        cmd = [
            'aria2c', url,
            '-o', os.path.basename(output_file),
            '-d', os.path.dirname(output_file),
            '-x', str(max_workers),
            '-s', str(max_workers),
            '--max-tries=5', '--retry-wait=3',
            '--user-agent=' + headers['User-Agent'],
            '--referer=' + BASE_URL
        ]

        subprocess.run(cmd, check=False)

        if os.path.exists(output_file):
            file_size = os.path.getsize(output_file) / (1024*1024)
            print(f"\n   ✅ Complete! ({file_size:.2f} MB)")
            return True
        return False
    except Exception as e:
        print(f"\n   ❌ Error: {e}")
        return False

def download_chunk(url, start, end, chunk_file, pbar):
    """Download a single chunk"""
    try:
        chunk_headers = headers.copy()
        chunk_headers['Range'] = f'bytes={start}-{end}'
        response = scraper.get(url, headers=chunk_headers, stream=True, timeout=30)

        if response.status_code not in [200, 206]:
            return False

        with open(chunk_file, 'wb') as f:
            for data in response.iter_content(chunk_size=8192):
                if data:
                    f.write(data)
                    pbar.update(len(data))
        return True
    except:
        return False

def download_with_chunks(url, output_file, episode_num):
    """Download with chunked/parallel downloading"""
    try:
        print(f"\n📥 Downloading Episode {episode_num} with chunks...")
        print(f"   File: {os.path.basename(output_file)}")

        head_response = scraper.head(url, headers=headers, timeout=10)

        if 'Content-Length' not in head_response.headers:
            return download_direct(url, output_file, episode_num)

        total_size = int(head_response.headers['Content-Length'])
        chunk_size = chunk_size_mb * 1024 * 1024

        chunks = []
        for i in range(0, total_size, chunk_size):
            start = i
            end = min(i + chunk_size - 1, total_size - 1)
            chunks.append((start, end))

        chunk_dir = f"{output_file}.chunks"
        os.makedirs(chunk_dir, exist_ok=True)
        chunk_files = []

        with tqdm(total=total_size, unit='B', unit_scale=True, desc="   ⏳ Downloading", ncols=80) as pbar:
            with ThreadPoolExecutor(max_workers=max_workers) as executor:
                futures = {}
                for idx, (start, end) in enumerate(chunks):
                    chunk_file = f"{chunk_dir}/chunk_{idx:04d}"
                    chunk_files.append(chunk_file)
                    future = executor.submit(download_chunk, url, start, end, chunk_file, pbar)
                    futures[future] = idx

                for future in as_completed(futures):
                    if not future.result():
                        shutil.rmtree(chunk_dir, ignore_errors=True)
                        return False

        print("   🔗 Merging chunks...")
        with open(output_file, 'wb') as outfile:
            for chunk_file in chunk_files:
                with open(chunk_file, 'rb') as infile:
                    shutil.copyfileobj(infile, outfile)

        shutil.rmtree(chunk_dir, ignore_errors=True)

        file_size = os.path.getsize(output_file) / (1024*1024)
        print(f"   ✅ Complete! ({file_size:.2f} MB)")
        return True
    except Exception as e:
        print(f"\n   ❌ Error: {e}")
        return False

def download_direct(url, output_file, episode_num):
    """Direct download with progress"""
    try:
        print(f"\n📥 Downloading Episode {episode_num}...")
        response = scraper.get(url, headers=headers, stream=True, timeout=30)
        total_size = int(response.headers.get('content-length', 0))

        with open(output_file, 'wb') as f:
            with tqdm(total=total_size, unit='B', unit_scale=True, desc="   ⏳", ncols=80) as pbar:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
                        pbar.update(len(chunk))

        file_size = os.path.getsize(output_file) / (1024*1024)
        print(f"   ✅ Complete! ({file_size:.2f} MB)")
        return True
    except Exception as e:
        print(f"\n   ❌ Error: {e}")
        return False

def download_episode(url, output_file, episode_num):
    """Main download with retry"""
    is_m3u8 = '.m3u8' in url

    if download_method == "yt-dlp":
        download_func = download_with_ytdlp
    elif download_method == "aria2" and not is_m3u8:
        download_func = download_with_aria2
    elif download_method == "chunks" and not is_m3u8:
        download_func = download_with_chunks
    else:
        download_func = download_with_ytdlp if is_m3u8 else download_with_chunks

    for attempt in range(1, max_retries + 1):
        if attempt > 1:
            print(f"\n   🔄 Retry {attempt}/{max_retries}...")
            time.sleep(3)

        if os.path.exists(output_file):
            os.remove(output_file)

        if download_func(url, output_file, episode_num):
            return True

    return False

print("✅ Download methods loaded")

# @title 📺 **Fetch Anime Info & Download** { display-mode: "form" }

print("\n" + "=" * 70)
print("🎬 ANIMEKAI EPISODE DOWNLOADER")
print("=" * 70)

print(f"\n🔍 Processing: {anime_url}")

anime_id, anime_title = get_anime_details(anime_url)

if not anime_id:
    raise Exception("❌ Could not extract anime ID")

print(f"✅ Anime ID: {anime_id}")
print(f"📺 Title: {anime_title}")

episode_list = get_episode_list(anime_id)

if not episode_list:
    raise Exception("❌ No episodes found!")

print(f"📋 Found {len(episode_list)} episode(s)")

# Determine episodes
if download_mode == "Single Episode":
    episodes_to_download = [ep for ep in episode_list if ep['number'] == single_episode]
elif download_mode == "Episode Range":
    episodes_to_download = [ep for ep in episode_list if start_episode <= ep['number'] <= end_episode]
else:
    episodes_to_download = episode_list

if not episodes_to_download:
    raise Exception("❌ No episodes match selection!")

print(f"📥 Will download {len(episodes_to_download)} episode(s)")

download_dir = f"downloads/{anime_title}"
os.makedirs(download_dir, exist_ok=True)

type_map = {"Hard Sub": "sub", "Soft Sub": "softsub", "Dub & S-Sub": "dub"}
prefer_type_id = type_map.get(prefer_type, "softsub")

downloaded_files = []
failed_episodes = []

for idx, episode in enumerate(episodes_to_download, 1):
    print(f"\n[{idx}/{len(episodes_to_download)}] Episode {episode['number']}")

    try:
        servers = get_video_servers(episode['token'])
        if not servers:
            failed_episodes.append(episode['number'])
            continue

        # Map preference types
        # Note: "dub" type on AnimeKai means Dub & S-Sub (dual audio with soft subs)
        type_map_search = {
            "Hard Sub": "sub",
            "Soft Sub": "softsub",
            "Dub & S-Sub": "dub"  # This is the dual audio option
        }
        prefer_type_id = type_map_search.get(prefer_type, "softsub")

        # Filter servers by preference
        matching_servers = [s for s in servers if s['type'] == prefer_type_id and s['server_name'] == prefer_server]

        # Fallback 1: Try any server with matching type
        if not matching_servers:
            matching_servers = [s for s in servers if s['type'] == prefer_type_id]

        # Fallback 2: Try any server with preferred server name
        if not matching_servers:
            matching_servers = [s for s in servers if s['server_name'] == prefer_server]

        # Fallback 3: Use first available server
        if not matching_servers:
            matching_servers = servers[:1]

        server = matching_servers[0]

        # Show what type we're actually downloading
        type_display = {
            "sub": "Hard Sub",
            "softsub": "Soft Sub",
            "dub": "Dub & S-Sub (Dual Audio)"
        }.get(server['type'], server['type'])

        print(f"   🎥 Server: {server['server_name']} | Type: {type_display}")

        video_url = get_video_url(server['server_id'], server['server_name'])
        if not video_url:
            failed_episodes.append(episode['number'])
            continue

        ep_num_str = f"{int(episode['number']):03d}"

        # Add type to filename for clarity
        type_suffix = {
            "sub": "HardSub",
            "softsub": "SoftSub",
            "dub": "DualAudio"
        }.get(server['type'], "")

        filename = f"{download_dir}/Episode_{ep_num_str}_{type_suffix}.mp4"

        if download_episode(video_url, filename, episode['number']):
            downloaded_files.append(filename)
        else:
            failed_episodes.append(episode['number'])

        time.sleep(2)
    except Exception as e:
        print(f"   ❌ Error: {e}")
        failed_episodes.append(episode['number'])

print("\n" + "=" * 70)
print("📊 DOWNLOAD SUMMARY")
print("=" * 70)
print(f"\n✅ Downloaded: {len(downloaded_files)} episode(s)")
if failed_episodes:
    print(f"❌ Failed: {', '.join(map(str, failed_episodes))}")
if downloaded_files:
    total_size = sum(os.path.getsize(f) for f in downloaded_files) / (1024*1024)
    print(f"💾 Total: {total_size:.2f} MB")
print("\n🎉 COMPLETE!")

In [ ]:

# 🎬 Video Episode Merger & Uploader
# Download ZIP file with video episodes, extract, merge them in order, and upload

# @title 🔧 **Install Dependencies** { display-mode: "form" }
print("📦 Installing required packages...")
!pip install -q natsort requests
!apt-get -qq install -y ffmpeg > /dev/null 2>&1
print("✅ All dependencies installed!\n")

# @title ⚙️ **Configuration** { display-mode: "form" }

#@markdown ### 📥 Download Settings
#@markdown Enter the direct download link (DDL) for your ZIP file:
zip_url = "https://example.com/videos.zip" #@param {type:"string"}

#@markdown Custom User-Agent (leave default if unsure):
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36" #@param {type:"string"}

#@markdown ---
#@markdown ### 🎬 Merge Settings
#@markdown Custom output name (leave empty to auto-detect from episodes):
custom_output_name = "" #@param {type:"string"}

#@markdown Video quality for merge:
merge_quality = "Copy Original (Fastest)" #@param ["Copy Original (Fastest)", "Re-encode High Quality", "Re-encode Compressed"]

#@markdown ---
#@markdown ### 📤 Upload Settings
upload_destination = "Both (GoFile + Google Drive)" #@param ["GoFile.io Only", "Google Drive Only", "Both (GoFile + Google Drive)", "None (Keep Local Only)"]

#@markdown Create ZIP of merged video for upload?
create_upload_zip = False #@param {type:"boolean"}

print("✅ Configuration set!")

# @title 📥 **Download ZIP File** { display-mode: "form" }
import requests
import os
import re
from urllib.parse import unquote, urlparse
from pathlib import Path

print("🔽 Starting download...")
print(f"🔗 URL: {zip_url[:60]}..." if len(zip_url) > 60 else f"🔗 URL: {zip_url}")

headers = {'User-Agent': user_agent}

try:
    response = requests.get(zip_url, headers=headers, stream=True, allow_redirects=True)
    response.raise_for_status()

    # Smart filename detection
    zip_filename = None

    # Method 1: Content-Disposition header
    if 'Content-Disposition' in response.headers:
        cd = response.headers['Content-Disposition']
        filenames = re.findall(r'filename\*?=["\']?(?:UTF-8\'\')?([^"\';]+)["\']?', cd)
        if filenames:
            zip_filename = unquote(filenames[0])
            print(f"📋 Filename from header: {zip_filename}")

    # Method 2: Final URL after redirects
    if not zip_filename:
        final_url = response.url
        url_path = urlparse(final_url).path
        zip_filename = os.path.basename(url_path)
        zip_filename = unquote(zip_filename)
        print(f"📋 Filename from URL: {zip_filename}")

    # Method 3: Extract meaningful name from URL
    if not zip_filename or zip_filename in ['', 'download', 'file']:
        # Try to extract from full URL path
        url_parts = [p for p in urlparse(zip_url).path.split('/') if p and p != 'download']
        if url_parts:
            zip_filename = url_parts[-1]
            zip_filename = unquote(zip_filename)

    # Ensure .zip extension
    if not zip_filename.lower().endswith('.zip'):
        if '.' not in zip_filename:
            zip_filename += '.zip'
        else:
            zip_filename = os.path.splitext(zip_filename)[0] + '.zip'

    # Clean filename (remove invalid characters)
    zip_filename = re.sub(r'[<>:"|?*\\]', '_', zip_filename)
    zip_filename = re.sub(r'[\x00-\x1f]', '', zip_filename)  # Remove control characters

    print(f"💾 Saving as: {zip_filename}")

    total_size = int(response.headers.get('content-length', 0))
    downloaded = 0

    with open(zip_filename, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)
                downloaded += len(chunk)
                if total_size:
                    percent = (downloaded / total_size) * 100
                    mb_downloaded = downloaded / (1024*1024)
                    mb_total = total_size / (1024*1024)
                    print(f"\r⏳ Progress: {percent:.1f}% ({mb_downloaded:.1f}/{mb_total:.1f} MB)", end='')

    print(f"\n✅ Downloaded: {zip_filename} ({downloaded / (1024*1024):.2f} MB)")

    # Extract base name for later use
    ZIP_BASE_NAME = os.path.splitext(zip_filename)[0]
    ZIP_BASE_NAME = re.sub(r'[_\-\s]+', ' ', ZIP_BASE_NAME).strip()

    print(f"📦 Base name extracted: '{ZIP_BASE_NAME}'")

except Exception as e:
    print(f"\n❌ Download failed: {str(e)}")
    import traceback
    print(traceback.format_exc())
    raise

# @title 📦 **Extract ZIP File** { display-mode: "form" }
import zipfile

extract_folder = "extracted_videos"
os.makedirs(extract_folder, exist_ok=True)

print(f"\n📂 Extracting to: {extract_folder}/")
print("⏳ Please wait...")

try:
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        file_list = zip_ref.namelist()
        total_files = len(file_list)

        print(f"📋 Found {total_files} file(s) in ZIP\n")

        # Extract with progress
        for idx, file in enumerate(file_list, 1):
            zip_ref.extract(file, extract_folder)
            if idx % 5 == 0 or idx == total_files:
                print(f"\r⏳ Extracting: {idx}/{total_files} files...", end='')

        print(f"\n\n📄 Extracted files:")
        video_count = 0
        for file in file_list:
            file_lower = file.lower()
            is_video = any(file_lower.endswith(ext) for ext in ['.mp4', '.mkv', '.avi', '.mov', '.flv', '.wmv', '.webm', '.m4v'])
            icon = "🎬" if is_video else "📄"
            print(f"  {icon} {file}")
            if is_video:
                video_count += 1

        print(f"\n✅ Extraction complete! Found {video_count} video file(s)")

except Exception as e:
    print(f"\n❌ Extraction failed: {str(e)}")
    raise

# @title 🔍 **Detect and Sort Episodes** { display-mode: "form" }
import re
from natsort import natsorted

def extract_episode_info(filename):
    """Enhanced episode detection with better pattern matching"""
    name = os.path.basename(filename)

    # Combined season and episode patterns (S01E01, S1E1, etc.)
    combined_patterns = [
        r'[Ss](\d+)[Ee](\d+)',  # S01E01, S1E1
        r'[Ss]eason[\s._-]*(\d+)[\s._-]*[Ee]pisode[\s._-]*(\d+)',  # Season 1 Episode 1
        r'[Ss]eason[\s._-]*(\d+)[\s._-]*[Ee][Pp][\s._-]*(\d+)',  # Season 1 Ep 1
        r'(\d+)[xX](\d+)',  # 1x01
    ]

    # Try combined patterns first
    for pattern in combined_patterns:
        match = re.search(pattern, name, re.IGNORECASE)
        if match:
            return int(match.group(1)), int(match.group(2))

    # Separate season patterns
    season_patterns = [
        r'[Ss]eason[\s._-]*(\d+)',
        r'[Ss](\d+)(?![Ee])',  # S1 but not followed by E
        r'Season[\s._-]*(\d+)',
    ]

    # Episode patterns
    episode_patterns = [
        r'[Ee]pisode[\s._-]*(\d+)',
        r'[Ee][Pp][\s._-]*(\d+)',
        r'[Ee](\d+)',
        r'Episode[\s._-]*(\d+)',
        r'[\s._-](\d{1,3})[\s._-]',  # Number surrounded by separators
        r'^(\d{1,3})[\s._-]',  # Number at start
        r'[\s._-](\d{1,3})\.',  # Number before extension
    ]

    season = None
    episode = None

    # Find season
    for pattern in season_patterns:
        match = re.search(pattern, name, re.IGNORECASE)
        if match:
            season = int(match.group(1))
            break

    # Find episode
    for pattern in episode_patterns:
        match = re.search(pattern, name, re.IGNORECASE)
        if match:
            ep_num = int(match.group(1))
            # Reasonable episode number (1-999)
            if 1 <= ep_num <= 999:
                episode = ep_num
                break

    return season, episode

# Find all video files
video_extensions = ['.mp4', '.mkv', '.avi', '.mov', '.flv', '.wmv', '.webm', '.m4v', '.ts', '.m2ts']
video_files = []

for root, dirs, files in os.walk(extract_folder):
    for file in files:
        if any(file.lower().endswith(ext) for ext in video_extensions):
            full_path = os.path.join(root, file)
            video_files.append(full_path)

if not video_files:
    print("❌ No video files found in the ZIP!")
    raise Exception("No video files detected")

print(f"🎬 Found {len(video_files)} video file(s)\n")

# Extract info and sort
video_info = []
for vf in video_files:
    season, episode = extract_episode_info(vf)
    video_info.append({
        'path': vf,
        'name': os.path.basename(vf),
        'season': season if season else 0,
        'episode': episode if episode else 0
    })

# Sort by season, then episode, then natural name
video_info.sort(key=lambda x: (x['season'], x['episode'], x['name']))

print("📋 **Detected Episode Order:**")
print("=" * 70)
for idx, info in enumerate(video_info, 1):
    s_info = f"S{info['season']:02d}" if info['season'] else "S??"
    e_info = f"E{info['episode']:02d}" if info['episode'] else "E??"
    size_mb = os.path.getsize(info['path']) / (1024*1024)
    print(f"{idx:2d}. [{s_info}{e_info}] {info['name'][:45]:<45} ({size_mb:.1f} MB)")
print("=" * 70)

# @title 🎞️ **Merge Videos** { display-mode: "form" }
import subprocess

print("\n🎬 Preparing to merge videos...")

# Create file list for ffmpeg
list_file = "filelist.txt"
with open(list_file, 'w', encoding='utf-8') as f:
    for info in video_info:
        # Escape single quotes for ffmpeg
        safe_path = info['path'].replace("'", "'\\''")
        f.write(f"file '{safe_path}'\n")

print(f"✅ Created merge list with {len(video_info)} video(s)")

# Determine output filename
if custom_output_name:
    output_name = custom_output_name
    if not output_name.lower().endswith('.mp4'):
        output_name += '.mp4'
else:
    # Auto-generate name
    seasons = [v['season'] for v in video_info if v['season'] > 0]
    episodes = [v['episode'] for v in video_info if v['episode'] > 0]

    base_name = ZIP_BASE_NAME

    if seasons and episodes:
        min_season = min(seasons)
        max_season = max(seasons)
        min_episode = min(episodes)
        max_episode = max(episodes)

        if min_season == max_season:
            output_name = f"{base_name} Season {min_season:02d} Episodes {min_episode:02d}-{max_episode:02d}.mp4"
        else:
            output_name = f"{base_name} S{min_season:02d}-S{max_season:02d} Ep{min_episode:02d}-{max_episode:02d}.mp4"
    else:
        output_name = f"{base_name} Merged Complete.mp4"

# Clean output name
output_name = re.sub(r'[<>:"|?*\\]', '_', output_name)
output_name = re.sub(r'\s+', ' ', output_name).strip()

print(f"\n📁 Output filename: {output_name}")

# Build ffmpeg command based on quality setting
if merge_quality == "Copy Original (Fastest)":
    cmd = [
        'ffmpeg', '-f', 'concat', '-safe', '0', '-i', list_file,
        '-c', 'copy', output_name, '-y'
    ]
    print("⚡ Mode: Fast merge (copy streams, no re-encoding)")
elif merge_quality == "Re-encode High Quality":
    cmd = [
        'ffmpeg', '-f', 'concat', '-safe', '0', '-i', list_file,
        '-c:v', 'libx264', '-crf', '18', '-preset', 'slow',
        '-c:a', 'aac', '-b:a', '192k',
        output_name, '-y'
    ]
    print("🎨 Mode: High quality re-encode (slower, best quality)")
else:  # Compressed
    cmd = [
        'ffmpeg', '-f', 'concat', '-safe', '0', '-i', list_file,
        '-c:v', 'libx264', '-crf', '23', '-preset', 'medium',
        '-c:a', 'aac', '-b:a', '128k',
        output_name, '-y'
    ]
    print("📦 Mode: Compressed re-encode (smaller file size)")

print("\n⏳ Merging videos... This may take a while.\n")

try:
    # Run ffmpeg
    process = subprocess.Popen(cmd, stderr=subprocess.PIPE, universal_newlines=True)

    # Parse ffmpeg output for progress
    duration_pattern = re.compile(r'Duration: (\d{2}):(\d{2}):(\d{2})')
    time_pattern = re.compile(r'time=(\d{2}):(\d{2}):(\d{2})')

    total_duration = None

    for line in process.stderr:
        # Get total duration
        if total_duration is None:
            dur_match = duration_pattern.search(line)
            if dur_match:
                h, m, s = map(int, dur_match.groups())
                total_duration = h * 3600 + m * 60 + s

        # Get current time
        time_match = time_pattern.search(line)
        if time_match and total_duration:
            h, m, s = map(int, time_match.groups())
            current_time = h * 3600 + m * 60 + s
            percent = (current_time / total_duration) * 100
            print(f"\r🎬 Progress: {percent:.1f}% ({current_time//60}:{current_time%60:02d} / {total_duration//60}:{total_duration%60:02d})", end='')

    process.wait()

    if process.returncode == 0:
        file_size = os.path.getsize(output_name) / (1024*1024)
        print(f"\n\n✅ **Merge Complete!**")
        print("=" * 70)
        print(f"📁 Output: {output_name}")
        print(f"💾 Size: {file_size:.2f} MB")
        print(f"🎬 Episodes: {len(video_info)}")
        print("=" * 70)

        MERGED_VIDEO = output_name
    else:
        print(f"\n❌ Merge failed with exit code {process.returncode}")
        raise Exception("FFmpeg merge failed")

except Exception as e:
    print(f"\n❌ Error during merge: {str(e)}")
    raise
finally:
    # Cleanup
    if os.path.exists(list_file):
        os.remove(list_file)

# @title 📦 **Create ZIP of Merged Video (Optional)** { display-mode: "form" }

if create_upload_zip:
    print("\n📦 Creating ZIP file of merged video...")

    zip_output = output_name.replace('.mp4', '.zip')

    import zipfile
    with zipfile.ZipFile(zip_output, 'w', zipfile.ZIP_DEFLATED, compresslevel=0) as zipf:
        print(f"⏳ Adding {output_name} to ZIP...")
        zipf.write(output_name, os.path.basename(output_name))

    zip_size = os.path.getsize(zip_output) / (1024*1024)
    print(f"✅ ZIP created: {zip_output} ({zip_size:.2f} MB)")

    UPLOAD_FILE = zip_output
else:
    UPLOAD_FILE = MERGED_VIDEO
    print("\n📄 Will upload video file directly (no ZIP)")

# @title 📤 **Upload Files** { display-mode: "form" }

def upload_to_gofile(filepath):
    """Upload file to GoFile.io"""
    try:
        print("\n🌐 GoFile.io Upload")
        print("-" * 50)

        # Get best server
        server_response = requests.get('https://api.gofile.io/servers', timeout=30)
        server_response.raise_for_status()
        server_data = server_response.json()

        if server_data['status'] != 'ok':
            print("❌ Failed to get GoFile server")
            return None

        server = server_data['data']['servers'][0]['name']
        print(f"📡 Server: {server}")

        # Updated endpoint
        upload_url = f'https://{server}.gofile.io/contents/uploadfile'

        file_size_mb = os.path.getsize(filepath) / (1024*1024)
        print(f"📦 File: {os.path.basename(filepath)} ({file_size_mb:.2f} MB)")
        print("⏳ Uploading... (this may take several minutes for large files)")

        with open(filepath, 'rb') as f:
            files_data = {'file': (os.path.basename(filepath), f, 'application/octet-stream')}
            response = requests.post(upload_url, files=files_data, timeout=7200)

        response.raise_for_status()

        result = response.json()
        if result['status'] == 'ok':
            download_page = result['data']['downloadPage']
            print("✅ Upload successful!")
            print(f"🔗 Link: {download_page}")
            return download_page
        else:
            print(f"❌ Upload failed: {result.get('message', 'Unknown error')}")
            return None

    except requests.exceptions.Timeout:
        print("❌ Upload timed out - file may be too large for GoFile")
        return None
    except requests.exceptions.JSONDecodeError:
        print("❌ Invalid response from GoFile - service may be down")
        return None
    except Exception as e:
        print(f"❌ GoFile error: {str(e)}")
        return None

def upload_to_gdrive(filepath):
    """Upload file to Google Drive"""
    try:
        print("\n☁️ Google Drive Upload")
        print("-" * 50)

        from google.colab import drive

        # Check if already mounted
        if not os.path.exists('/content/drive/MyDrive'):
            drive.mount('/content/drive', force_remount=False)
            print("✅ Google Drive mounted!")
        else:
            print("✅ Google Drive already mounted!")

        destination = '/content/drive/MyDrive/Merged_Videos/'
        os.makedirs(destination, exist_ok=True)

        dest_path = os.path.join(destination, os.path.basename(filepath))

        # Check if file exists
        if not os.path.exists(filepath):
            print(f"❌ Source file not found: {filepath}")
            return None

        file_size_mb = os.path.getsize(filepath) / (1024*1024)
        print(f"📦 File: {os.path.basename(filepath)} ({file_size_mb:.2f} MB)")
        print(f"⏳ Copying to Google Drive...")

        import shutil
        shutil.copy2(filepath, dest_path)

        print("✅ Upload successful!")
        print(f"📁 Location: MyDrive/Merged_Videos/{os.path.basename(filepath)}")
        return dest_path

    except Exception as e:
        print(f"❌ Google Drive error: {str(e)}")
        import traceback
        print(traceback.format_exc())
        return None

# Execute uploads based on user selection
print("\n" + "=" * 70)
print("📤 UPLOAD PROCESS")
print("=" * 70)

# Track upload results
gofile_link = None
gdrive_path = None

if upload_destination == "GoFile.io Only":
    gofile_link = upload_to_gofile(UPLOAD_FILE)

elif upload_destination == "Google Drive Only":
    gdrive_path = upload_to_gdrive(UPLOAD_FILE)

elif upload_destination == "Both (GoFile + Google Drive)":
    gofile_link = upload_to_gofile(UPLOAD_FILE)
    gdrive_path = upload_to_gdrive(UPLOAD_FILE)

else:  # None
    print("\n📁 Upload skipped - file saved locally")
    print(f"📄 Location: /content/{UPLOAD_FILE}")

# Summary
print("\n" + "=" * 70)
print("🎉 **ALL DONE!**")
print("=" * 70)
print(f"\n📊 Summary:")
print(f"  • Videos merged: {len(video_info)}")
print(f"  • Output file: {output_name}")
print(f"  • File size: {os.path.getsize(MERGED_VIDEO) / (1024*1024):.2f} MB")

if gofile_link:
    print(f"\n🔗 GoFile.io Link:")
    print(f"   {gofile_link}")

if gdrive_path:
    print(f"\n📁 Google Drive:")
    print(f"   {gdrive_path}")

if not gofile_link and not gdrive_path and upload_destination != "None (Keep Local Only)":
    print(f"\n⚠️ Note: Some uploads may have failed. Check error messages above.")

print("\n✨ Process complete!")